In [3]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.yaml"
# config_file = "../../../config/afun.yaml"

In [4]:
from bokeh.io import output_notebook
import malariagen_data
from IPython.display import Markdown
from selection_atlas.setup import AtlasSetup
from selection_atlas.page_utils import AtlasPageUtils

# Initialise the atlas setup.
setup = AtlasSetup(config_file)
page_utils = AtlasPageUtils(setup=setup)

# N.B., do not add the "remove-output" tag to this cell!!! If you do,
# the bokeh javascript libraries will not get loaded in the generated
# HTML page. The call to output_notebook() injects javascript in the
# cell output which triggers the bokeh javascript libraries to be loaded
# in the page.
output_notebook(hide_banner=True)

# Methods

## Data sources

In [5]:
df_samples = setup.sample_metadata()
countries = df_samples["country"].unique()

In [6]:
Markdown(f"""
This report analyses genome variation data from the 
{{term}}`Malaria Vector Genome Observatory`. See Table 1 below for a 
complete list of the sample sets used in the current analysis version, 
with information about the corresponding contributors, data releases 
and citations. These sample sets provide data for a total of
{len(df_samples):,} mosquitoes sampled from {len(countries)} countries. 
""")


This report analyses genome variation data from the 
{term}`Malaria Vector Genome Observatory`. See Table 1 below for a 
complete list of the sample sets used in the current analysis version, 
with information about the corresponding contributors, data releases 
and citations. These sample sets provide data for a total of
4,878 mosquitoes sampled from 25 countries. 


In [7]:
page_utils.style_data_sources(
    df_samples=df_samples,
    caption="Table 1. Data sources included in the current analysis version.",
)

Sample Set,Study,Contributor,Data Release,Citation
AG1000G-AO,AG1000G-AO,Joao Pinto,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2017
AG1000G-BF-A,AG1000G-BF-1,Austin Burt,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2017
AG1000G-BF-B,AG1000G-BF-1,Austin Burt,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021
AG1000G-BF-C,AG1000G-BF-2,Nora Besansky,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021
AG1000G-CD,AG1000G-CD,David Weetman,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021
AG1000G-CF,AG1000G-CF,Alessandra della Torre,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021
AG1000G-CI,AG1000G-CI,David Weetman,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2020
AG1000G-CM-A,AG1000G-CM-1,Nora Besansky,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2017
AG1000G-CM-B,AG1000G-CM-2,Nora Besansky,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021
AG1000G-CM-C,AG1000G-CM-3,Brad White,Ag3.0,Anopheles gambiae 1000 Genomes Consortium 2021


In [8]:
Markdown(f"""
Sample metadata, unphased SNP calls, and phased SNP haplotypes were retrieved from 
the {{term}}`Malaria Vector Genome Observatory` cloud data repository hosted in 
Google Cloud Storage (GCS) via the {{term}}`MalariaGEN Python API` version 
{malariagen_data.__version__}.
""")


Sample metadata, unphased SNP calls, and phased SNP haplotypes were retrieved from 
the {term}`Malaria Vector Genome Observatory` cloud data repository hosted in 
Google Cloud Storage (GCS) via the {term}`MalariaGEN Python API` version 
15.0.1.


## Sample inclusion and grouping into cohorts

Samples were considered for inclusion if they met the following criteria:

In [15]:
def human_readable_list(x):
    x = [str(v) for v in x]
    if len(x) > 1:
        output = ", ".join(x[:-1]) + " or " + x[-1]
    else:
        output = x[0]
    return output


readable_taxa = human_readable_list(setup.taxa)

In [16]:
Markdown(f"""
* Gender assigned as female via comparison of sequence coverage on autosomes and sex 
  chromosomes. 
* Taxon assigned as {readable_taxa} via principle components analysis of 
  genomic data from Chromosome 3 and comparison with reference samples 
  with known taxon assignments.
""")


* Gender assigned as female via comparison of sequence coverage on autosomes and sex 
  chromosomes. 
* Taxon assigned as gambiae, coluzzii, arabiensis or bissau via principle components analysis of 
  genomic data from Chromosome 3 and comparison with reference samples 
  with known taxon assignments.


In [17]:
gdf_cohorts = page_utils.gdf_cohorts

After filtering according to these inclusion criteria, samples were grouped into cohorts by taxon, location of sampling and date of sampling. Samples were grouped spatially if their collection locations were within the same level 2 administrative unit, according to geoBoundaries version 5.0.0. Samples were grouped temporally if their collection dates were within the same quarter (3 month period) where possible, except in a small number of cases where metadata were only available on year of collection.

In [18]:
Markdown(f"""
Cohorts were excluded from the analysis if the sample size was 
less than {setup.min_cohort_size}. Cohorts with more than 
{setup.max_cohort_size} samples were randomly downsampled for
computational efficiency. Cohorts were also excluded from the 
analysis if they failed H12 or G123 window size calibration
(see below). After applying these filters, a total of 
{len(gdf_cohorts)} cohorts were retained for analysis (Table 2).
""")


Cohorts were excluded from the analysis if the sample size was 
less than 15. Cohorts with more than 
100 samples were randomly downsampled for
computational efficiency. Cohorts were also excluded from the 
analysis if they failed H12 or G123 window size calibration
(see below). After applying these filters, a total of 
61 cohorts were retained for analysis (Table 2).


In [19]:
page_utils.style_cohorts_table(
    gdf_cohorts,
    caption="Table 2. Cohorts selected for genome-wide selection scan analyses.",
    url_prefix="",
)

Cohort,Country,Region,District,Taxon,Year,Quarter,Sample Size
Angola / Luanda / coluzzii / 2009 / Q2,Angola,Luanda,Luanda,coluzzii,2009,2,77
Burkina Faso / Comoe / coluzzii / 2011,Burkina Faso,Cascades,Comoe,coluzzii,2011,,18
Burkina Faso / Comoe / coluzzii / 2012,Burkina Faso,Cascades,Comoe,coluzzii,2012,,63
Burkina Faso / Comoe / coluzzii / 2015,Burkina Faso,Cascades,Comoe,coluzzii,2015,,33
Burkina Faso / Comoe / coluzzii / 2016,Burkina Faso,Cascades,Comoe,coluzzii,2016,,53
Burkina Faso / Houet / coluzzii / 2012 / Q3,Burkina Faso,Hauts-Bassins,Houet,coluzzii,2012,3,78
Burkina Faso / Houet / coluzzii / 2014 / Q3,Burkina Faso,Hauts-Bassins,Houet,coluzzii,2014,3,32
Burkina Faso / Houet / gambiae / 2012 / Q3,Burkina Faso,Hauts-Bassins,Houet,gambiae,2012,3,73
Burkina Faso / Houet / gambiae / 2014 / Q3,Burkina Faso,Hauts-Bassins,Houet,gambiae,2014,3,41
Benin / Djougou / coluzzii / 2017 / Q2,Benin,Donga,Djougou,coluzzii,2017,2,78


## H12 and G123 window size calibration

Both H12 ([Garud et al. 2015](https://pmc.ncbi.nlm.nih.gov/articles/PMC4338236/)) and G123 ([Harris et al. 2018](https://pmc.ncbi.nlm.nih.gov/articles/PMC6283157/)) are statistical methods for performing {term}`genome-wide selection scans` which rely on dividing data into windows along the {term}`genome`. Typically the size of these windows is set to a fixed number of polymorphic sites ({term}`SNP`s). I.e., all windows contain data ─ either phased haplotypes or unphased genotypes ─ for the same number of {term}`SNP`s. In order to detect recent selective sweeps, the size of these windows needs to be chosen so that windows are generally larger than the normal genetic distance over which linkage disequlibrium (LD) decays to background levels in the absence of recent positive selection. Therefore, in windows which are unaffected by recent selective sweeps, genetic diversity will be high and thus the values of the selection statistics will be low. Conversely, in windows affected by recent selective sweeps, linkage disequilibrium will extend over a longer genetic distance spanning multiple windows, so that genetic diversity within those windows is low and thus values of selection statistics will be high. In other words, the choice of window size affects the signal to noise ratio for selection scans using H12 and G123 statistics. If windows are too small, results are dominated by background noise. If windows are too large, noise is minimal but power to detect recent selection signals is reduced.

This decision regarding an appropriate window size needs to be made independently for each cohort of samples over which a selection scan will be performed. This is because different source populations may have different demographic histories, and this in turn may alter the genetic distance over which LD decays in the absence of positive selection. Previous studies have used various demographic inference methods to try to infer key demographic parameters for each cohort being analysed, then use these parameters to inform the decision of window size. In practice, this approach presents a number of challenges. Firstly, inference of demographic parameters is difficult, and even state of the art inference methods may reach inaccurate conclusions. Secondly, running demographic inference methods can be computationally demanding, and this becomes impractical for large numbers of cohorts.

In [23]:
Markdown(f"""
For these reasons we have taken an empirical approach to window size calibration for H12 
and G123 scans, designed to reach a good signal to noise ratio. 

For each cohort, we compute H12 over contig {setup.h12_calibration_contig} for multiple window 
sizes of {human_readable_list(setup.h12_calibration_window_sizes)} SNPs. We then compute the 95th 
percentile of statistic values over all windows. We choose the smallest window size for which the 
95th percentile is below {setup.h12_calibration_threshold}. This means that any window with a
statistic value above this threshold will be in the top 5% of windows.

Similarly, we compute G123 over contig {setup.g123_calibration_contig} for multiple window 
sizes of {human_readable_list(setup.g123_calibration_window_sizes)} SNPs. We then compute the 95th 
percentile of statistic values over all windows. We choose the smallest window size for which the 
95th percentile is below {setup.g123_calibration_threshold}.

""")


For these reasons we have taken an empirical approach to window size calibration for H12 
and G123 scans, designed to reach a good signal to noise ratio. 

For each cohort, we compute H12 over contig 3RL for multiple window 
sizes of 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000 or 10000 SNPs. We then compute the 95th 
percentile of statistic values over all windows. We choose the smallest window size for which the 
95th percentile is below 0.08. This means that any window with a
statistic value above this threshold will be in the top 5% of windows.

Similarly, we compute G123 over contig 3RL for multiple window 
sizes of 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1200, 1400, 1600, 1800, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 6000, 7000, 8000, 9000 or 10000 SNPs. We then compute the 95th 
percentile of statistic values over all windows. We choose the smallest window size for which the 
95th percentile is below 0.08.



TODO how was window-size calibration done?

TODO after calibration, some cohorts removed if cannot get a window-size.

## H12 genome-wide selection scans

TODO

## G123 genome-wide selection scans

TODO

## IHS genome-wide selection scans

TODO

## Automated detection of selection signals

TODO

## Identification of selection alerts

TODO

## Web report generation

TODO